
Inteligencia Artificial I
Práctica 1.
(Sesión 3) Búsqueda heurística
El objetivo del puzle MovIX es utilizar el mínimo número de movimientos para
alinear (horizontal, vertical o diagonal) en un tablero NxN un conjunto X (X<= N) de
fichas con diferentes habilidades de movimiento.
Las celdas del tablero inicial pueden estar vacías o contener una ficha. El jugador
puede mover una ficha en cada turno y dos fichas no pueden ocupar la misma
casilla al mismo tiempo.
Hay varios tipos de fichas que tienen distintas habilidades de movimiento:
S- Saltadora: puede saltar sobre una ficha adyacente (horizontal o vertical)
y aterrizar en la siguiente celda vacía. No puede moverse si no es saltando
sobre una ficha, es decir, no se puede saltar sobre celdas vacías, ni sobre
dos fichas, ni sobre un muro (si se incluyen).
L- Lenta: puede moverse únicamente a una celda adyacente vacía en
cualquier dirección (horizontal, vertical o diagonal).
V- Vertical: puede moverse verticalmente a cualquier celda vacía en la
misma columna sin saltar fichas.
H- Horizontal: puede moverse horizontalmente a cualquier celda vacía en
la misma fila sin saltar fichas. Ejemplo: Si está en (1, 2) y toda la fila 1 está
vacía podría moverse a (1, 1), (1, 3), (1, 4), (1, 0).
La siguiente configuración es un ejemplo de la posición inicial de las fichas (al
inicio se coloca aleatoriamente un número dado (Y) de fichas de cada tipo. En el
ejemplo Y=1:
Se pide:
o Representa el problema en AIMA para resolverlo con búsqueda
heurística. Analiza el tamaño y forma del espacio de estados.
o Define varias H’ para el problema y analízalas comparando su
rendimiento al resolver distintos estados iniciales y distinto número
de fichas iniciales.
Belén Díaz Agudo
Inteligencia Artificial I
o Realiza pruebas con distintos tamaños de tablero N, de línea X, y
número de fichas iniciales de cada tipo (Y).
o Elige de forma justificada cual es la mejor h’ para este problema
o Compara la búsqueda heurística con algún algoritmo de búsqueda
ciega para medir las ventajas del uso de la heurística.
o Añade un nuevo tipo de casillas M muro sobre las que no se puede
saltar, es decir, un muro detiene el movimiento de las fichas V
vertical, H horizontal. Prueba el efecto que tiene en la resolución de
tableros usando las mismas heurísticas.
Ideas para definir heurísticas:
 Ten en cuenta si hay fichas ya alineadas y cuántas fichas adicionales se
necesitarían para completar la alineación de X fichas.
 Considera los conflictos entre fichas que impiden la alineación. Cada ficha
que está en la línea de alineación pero que no puede moverse (por ser de
un tipo que no permite el movimiento adecuado) suma un costo a la
heurística. Cuenta cuántas fichas están en la misma línea que no pueden
alinearse debido a sus restricciones de movimiento. Cada conflicto podría
sumar un valor (por ejemplo, 1 por cada conflicto).
 Combina las heurísticas anteriores para dar un valor más integral. Podrías
ponderar la distancia de Manhattan y la alineación parcial para crear una
heurística más completa. 

In [96]:
## Copiamos aquí la definición de la clase Problem de la librería AIMA para facilitar la explicación.
from search import *

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [97]:

import numpy as np

class MovIX(Problem):
    def __init__(self, initial, numpieces, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        dtype = [('piece', '<U1'), ('location', '<i4', 2)]
        self.initial = np.zeros(numpieces, dtype=dtype)
        self.InicializarEstadoInicial(initial)
        self.X = numpieces
        self.N = len(initial)
        self.goal = goal
        print("El estado inicial es objetivo? ",self.goal_test(self.initial))

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state)-> bool:
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""

        '''La función estudia si los puntos donde están colocados las piezas forman una semirrecta sin huecos.
        Para ello calcula las distancias de todos los puntos en las dos coordenadas, así como
        la pendiente de todos los puntos respecto al primero de los puntos.
        Si la pendiente es constante y dentro de los valores posibles según las restricciones del estado objetivo
        (m=+-1 para las diagonales, 0 para las horizontales o Inf para las verticales), los puntos están alineados.
        Además, si la distancia entre los valores no excede el número de puntos, se tiene una solución (ya que al no poder haber repetidos,
        X puntos alineados que no exceden la distancia X al primero por fuerza forman la semirrecta sin huecos)'''

        if self.X <= 1: return True #Si sólo hay un punto la semirrecta es trivial

        posiciones = state['location']#Tomamos solo las posiciones de las piezas

        firstPiece, secondPiece = posiciones[0], posiciones[1]#Calculamos la pendiente entre las dos primeras
       
        #En python no existe el do-while :(

        firsti, firstj = firstPiece

        secondi, secondj = secondPiece

        # Si la recta es vertical la pendiente será infinita y no queremos que salte un warning
        np.seterr(divide='ignore', invalid='ignore')

        deltaj, deltai = (secondj - firstj), (secondi - firsti)

        currentSlope = np.divide(deltai, deltaj)#Pendiente respecto a la primera pieza

        if abs(deltai) >= self.X or abs(deltaj) >= self.X:return False #No basta con estar alineados, no puede haber huecos en la semirrecta
        if abs(currentSlope) != 1 and currentSlope != 0 and not np.isinf(currentSlope): return False #La pendiente de la semirrecta solo puede tomar los valores 0, +-1 y Inf

        for i in range(2,self.X):
            secondi, secondj = posiciones[i]
            deltaj, deltai = (secondj - firstj), (secondi - firsti)
            slope = np.divide(deltai, deltaj)#Pendiente respecto a la primera pieza              
            if slope != currentSlope:return False #La pendiente de toda la recta debe ser la misma
            if abs(deltai) >= self.X or abs(deltaj) >= self.X:return False #No basta con estar alineados, no puede haber huecos en la semirrecta

        # Es una buena práctia restablecer el warning de dividir entre cero
        np.seterr(divide='warn', invalid='warn')

        #Si nos hemos encontrado problemas tenemos nuestra semirrecta
        return True     
           
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError
    
    def leer_tablero_archivo_sin_espacios(ruta_archivo):
        with open(ruta_archivo, 'r') as archivo:
            # Leer todas las líneas y convertirlas en una matriz de caracteres
            tablero = [list(linea.strip()) for linea in archivo]
        
        return tablero
    
    def InicializarEstadoInicial(self,tablero):
        cont = 0
        N = len(tablero)
        for i in range(N):
            for j in range(N):
                var = tablero[i][j]
                if var != "_":
                    self.initial[cont][0]= var
                    self.initial[cont][1]= (i, j)
                    cont += 1

    

    

In [98]:
tableroEjemplo = [
    ['_', '_', '_', '_', '_'],
    ['_', 'S', '_', 'H', '_'],
    ['_', '_', 'L', '_', '_'],
    ['V', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_']
]


tableroVertical = [
    ['S', '_', '_', '_', '_'],
    ['V', '_', '_', '_', '_'],
    ['L', '_', '_', '_', '_'],
    ['H', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_']
]

tableroVertical2 = [
    ['_', '_', '_', '_', '_'],
    ['_', 'V', '_', '_', '_'],
    ['_', 'L', '_', '_', '_'],
    ['_', 'H', '_', '_', '_'],
    ['_', 'S', '_', '_', '_']
]

tableroHorizontal = [
    ['S', 'V', 'L', 'H', '_'],
    ['_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_']
]

tableroHorizontal2 = [
    ['_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_'],
    ['_', 'V', 'L', 'H', 'S'],
    ['_', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_']
]

tableroDiagonal = [
    ['S', '_', '_', '_', '_'],
    ['_', 'V', '_', '_', '_'],
    ['_', '_', 'L', '_', '_'],
    ['_', '_', '_', 'H', '_'],
    ['_', '_', '_', '_', '_']
]

tableroDiagonal2 = [
    ['_', '_', '_', '_', 'H'],
    ['_', '_', '_', 'V', '_'],
    ['_', '_', 'L', '_', '_'],
    ['_', 'S', '_', '_', '_'],
    ['_', '_', '_', '_', '_']
]

p = MovIX(tableroEjemplo,4)
p = MovIX(tableroVertical,4)
p = MovIX(tableroVertical2,4)
p = MovIX(tableroHorizontal,4)
p = MovIX(tableroHorizontal2,4)
p = MovIX(tableroDiagonal,4)
p = MovIX(tableroDiagonal2,4)

El estado inicial es objetivo?  False
El estado inicial es objetivo?  True
El estado inicial es objetivo?  True
El estado inicial es objetivo?  True
El estado inicial es objetivo?  True
El estado inicial es objetivo?  True
El estado inicial es objetivo?  True
